In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime

In [2]:
name = []
mileage = []
dealer_name = []
rating = []
rating_count = []
price = []

for i in range(1,41):
    
    # Website in a variable
    website = 'https://www.cars.com/shopping/results/?page=' + str(i) + '&page_size=20&dealer_id=&keyword=&list_price_max=&list_price_min=&makes[]=bmw&maximum_distance=all&mileage_max=&models[]=bmw-335&sort=best_match_desc&stock_type=used&year_max=2011&year_min=2007&zip='

    # Request to website
    response = requests.get(website)
    
    # Soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Results
    results = soup.find_all('div', {'class':'vehicle-card'})
    
    
    for result in results:
    
        # name
        try:
            name.append(result.find('h2').get_text())
        except:
            name.append('n/a')


        # mileage
        try:
            mileage.append(result.find('div', {'class':'mileage'}).get_text())
        except:
            mileage.append('n/a')

        # dealer_name
        try:
            dealer_name.append(result.find('div', {'class': 'dealer-name'}).get_text().strip())
        except:
            dealer_name.append('n/a')

        # rating
        try:
            rating.append(result.find('span',{'class':'sds-rating__count'}).get_text())
        except:
            rating.append('n/a')


        # rating_count
        try:
            rating_count.append(result.find('span', {'class': 'sds-rating__link'}).get_text())
        except:
            rating_count.append('n/a')


        # price
        try:
            price.append(result.find('span', {'class': 'primary-price'}).get_text())
        except:
            price.append('n/a')

In [3]:
bmw_df = pd.DataFrame({'Name':name, 'Mileage': mileage,'Dealer Name': dealer_name,
                          'Rating': rating, 'Rating Count': rating_count, 'Price':price})

In [4]:
bmw_df

,Name,Mileage,Dealer Name,Rating,Rating Count,Price
0,2009 BMW 335 i,"82,481 mi.",Caspian Auto Motors,3.7,(170 reviews),"$14,677"
1,2011 BMW 335 i xDrive,"61,618 mi.","Guess Buick GMC / Guess Ford, Inc.",4.9,(74 reviews),"$18,200"
2,2011 BMW 335 i xDrive,"86,524 mi.",Loyalty Volkswagen,4.6,(206 reviews),"$16,598"
3,2010 BMW 335 i,"48,438 mi.",Carvana-Touchless Delivery To Your Home,4.2,"(8,898 reviews)","$21,590"
4,2011 BMW 335 i,"40,350 mi.",McGrath Lexus of Westmont,4.8,"(1,892 reviews)","$25,988"
...,...,...,...,...,...,...
341,2007 BMW 335 xi,"128,442 mi.",Jet's Auto Sales,2.9,(37 reviews),"$12,995"
342,2008 BMW 335 i,"65,100 mi.",Jet's Auto Sales,2.9,(37 reviews),"$14,995"
343,2008 BMW 335 xi,"107,298 mi.",East Coast Auto Source,2.5,(112 reviews),"$5,595"
344,2011 BMW 335 i,"77,654 mi.",Auto Aves,4.8,"(1,473 reviews)","$17,980"


### Data Cleaning

In [5]:
bmw_df['Rating Count'] = bmw_df['Rating Count'].apply(lambda x: x.strip('reviews)').strip('('))

# JUNE 2nd, 2022
## Further data cleaning prep for SQL

### Parse year

In [6]:
bmw_df['Year'] = bmw_df['Name'].apply(lambda x: x.split(' ')[0])

In [7]:
bmw_df

,Name,Mileage,Dealer Name,Rating,Rating Count,Price,Year
0,2009 BMW 335 i,"82,481 mi.",Caspian Auto Motors,3.7,170,"$14,677",2009
1,2011 BMW 335 i xDrive,"61,618 mi.","Guess Buick GMC / Guess Ford, Inc.",4.9,74,"$18,200",2011
2,2011 BMW 335 i xDrive,"86,524 mi.",Loyalty Volkswagen,4.6,206,"$16,598",2011
3,2010 BMW 335 i,"48,438 mi.",Carvana-Touchless Delivery To Your Home,4.2,"8,898","$21,590",2010
4,2011 BMW 335 i,"40,350 mi.",McGrath Lexus of Westmont,4.8,"1,892","$25,988",2011
...,...,...,...,...,...,...,...
341,2007 BMW 335 xi,"128,442 mi.",Jet's Auto Sales,2.9,37,"$12,995",2007
342,2008 BMW 335 i,"65,100 mi.",Jet's Auto Sales,2.9,37,"$14,995",2008
343,2008 BMW 335 xi,"107,298 mi.",East Coast Auto Source,2.5,112,"$5,595",2008
344,2011 BMW 335 i,"77,654 mi.",Auto Aves,4.8,"1,473","$17,980",2011


### Remove 'mi' from mileage

In [8]:
bmw_df['Mileage'] = bmw_df['Mileage'].apply(lambda x: x.split(' ')[0])

### Remove \$ from price


In [9]:
'''
Reasoning for the -1 for future reference:
because if happens that occurrence is not in the string you'll get "IndexError: list index out of range".

Therefore -1 will not get any harm cause number of occurrences is already set to one.
'''

bmw_df['Price'] = bmw_df['Price'].apply(lambda x: x.split('$')[-1])


In [10]:
bmw_df

,Name,Mileage,Dealer Name,Rating,Rating Count,Price,Year
0,2009 BMW 335 i,"82,481",Caspian Auto Motors,3.7,170,"14,677",2009
1,2011 BMW 335 i xDrive,"61,618","Guess Buick GMC / Guess Ford, Inc.",4.9,74,"18,200",2011
2,2011 BMW 335 i xDrive,"86,524",Loyalty Volkswagen,4.6,206,"16,598",2011
3,2010 BMW 335 i,"48,438",Carvana-Touchless Delivery To Your Home,4.2,"8,898","21,590",2010
4,2011 BMW 335 i,"40,350",McGrath Lexus of Westmont,4.8,"1,892","25,988",2011
...,...,...,...,...,...,...,...
341,2007 BMW 335 xi,"128,442",Jet's Auto Sales,2.9,37,"12,995",2007
342,2008 BMW 335 i,"65,100",Jet's Auto Sales,2.9,37,"14,995",2008
343,2008 BMW 335 xi,"107,298",East Coast Auto Source,2.5,112,"5,595",2008
344,2011 BMW 335 i,"77,654",Auto Aves,4.8,"1,473","17,980",2011


### Remove comas from Mileage and Price

In [11]:
bmw_df['Mileage'] = bmw_df['Mileage'].apply(lambda x: x.replace(',', ''))

In [12]:
bmw_df['Price'] = bmw_df['Price'].apply(lambda x: x.replace(',', ''))

## Save CSV 

In [13]:
filename = 'data_scrapped_' + datetime.today().strftime('%Y-%m-%d')
bmw_df.to_csv(filename, index=False)